In [1]:
import numpy as np
import pandas as pd
from ipywidgets import FloatProgress
from IPython.display import display
import timeit
from datetime import timedelta
from pathlib import Path
import datetime as dt 
import matplotlib.pyplot as plt
from collections import namedtuple
from scipy.io import loadmat
import scipy.spatial.distance as dist

mat = loadmat('spamData.mat')  # load mat-file
# mat = loadmat('../input/spamdata/spamData.mat')
# sorted(mat.keys())
x_test = mat['Xtest']  # variable in mat file
x_train = mat['Xtrain']  # variable in mat file
y_test = mat['ytest']  # variable in mat file
y_train = mat['ytrain']  # variable in mat file

In [2]:
# data preprocessing: binarization and log transformation
from sklearn import preprocessing

def logTrans(data):
    return np.log(data + 0.1)

xtrain = preprocessing.binarize(x_train)
xtest = preprocessing.binarize(x_test)
ytrain = y_train.flatten()
ytest = y_test.flatten()
xtest_log = logTrans(x_test)
xtrain_log = logTrans(x_train)

In [6]:
# let N1 be # of spam emails, N be total # of emails
N1train = 0
Ntrain = 0
for row in y_train:
#     print (row)
    if row == 1:
        N1train += 1
    Ntrain += 1
N0train = Ntrain - N1train
print (N1train, Ntrain, N1train/Ntrain)
MLE = N1train / Ntrain
MLE_neg = 1 - MLE
MLE_log = np.log(MLE)
MLE_neg_log = np.log(MLE_neg)

# # feature likelihood from training data
# feature_likelihood = np.mean(xtrain[ytrain == 1], axis = 0)
# print (feature_likelihood)

1240 3065 0.40456769983686786


In [13]:
def sum_x_train(spammed):
    return np.sum(xtrain[ytrain == spammed], axis = 0).astype('uint16')
def sum_y_train(spammed):
    return len(ytrain[ytrain == spammed])
def posterior_predictive_distribution (spammed, alpha):
    return ((sum_x_train(spammed)+alpha)/(sum_y_train(spammed)+2*alpha))
def classify(email, alpha):
    feature_count = len(email)
    
    param_spam = 1
    param_non_spam = 1
    
    temp_post_dist_0 = posterior_predictive_distribution(0, alpha)
    temp_post_dist_1 = posterior_predictive_distribution(1, alpha)
    
    for i in np.arange(feature_count):
#         param_spam += (email[i] == 1) * np.log(temp_post_dist_1) + \
#         (email[i] == 0) * np.log(1-temp_post_dist_1)
#         param_non_spam += (email[i] == 1) * np.log(temp_post_dist_0) + \
#         (email[i] == 0) * np.log(1-temp_post_dist_0)
        
#     prob_spam = MLE_log + param_spam
#     prob_non_spam = MLE_neg_log + param_non_spam
        if (email[i] == 1):
            param_spam = param_spam * temp_post_dist_1[i]
            param_non_spam = param_non_spam * temp_post_dist_0[i]
        if (email[i] == 0):
            param_spam = param_spam * (1 - temp_post_dist_1[i])
            param_non_spam = param_non_spam * (1 - temp_post_dist_0[i])
#         param_spam = param_spam * (temp_post_dist_1 ** (email[i] == 1)) * ((1-temp_post_dist_1) ** (email[i] == 0))
#         param_non_spam = param_non_spam * (temp_post_dist_0 ** (email[i] == 1)) * ((1-temp_post_dist_0) ** (email[i] == 0))
        
    prob_spam = MLE * param_spam
    prob_non_spam = MLE_neg * param_non_spam

    return (prob_spam > prob_non_spam).astype('uint8')
def calc_error_rate_test(alpha):
    error_count = 0
    for count, email in enumerate(xtest):
        if (classify(email, alpha) != ytest[count]):
            error_count += 1
    return error_count/len(xtest)
def calc_error_rate_train(alpha):
    error_count = 0
    for count, email in enumerate(xtrain):
        if (classify(email, alpha) != ytrain[count]):
            error_count += 1
    return error_count/len(xtrain) * 100

In [14]:
# compute error rate for test data with different alpha values
alphas = np.arange(0,100.5,0.5)
error_rate_test = []
error_rate_train = []

for alpha in alphas:
    error_rate_test.append(calc_error_rate_test(alpha))
    print ("alpha = ", alpha, "computed")
    print (error_rate_test)
print (error_rate_test)

for alpha in alphas:
    error_rate_train.append(calc_error_rate_train(alpha))
    print ("alpha = ", alpha, "computed")
print (error_rate_train)

alpha =  0.0 computed
[0.12044270833333333]
alpha =  0.5 computed
[0.12044270833333333, 0.12174479166666667]
alpha =  1.0 computed
[0.12044270833333333, 0.12174479166666667, 0.12369791666666667]
alpha =  1.5 computed
[0.12044270833333333, 0.12174479166666667, 0.12369791666666667, 0.12434895833333333]
alpha =  2.0 computed
[0.12044270833333333, 0.12174479166666667, 0.12369791666666667, 0.12434895833333333, 0.125]
alpha =  2.5 computed
[0.12044270833333333, 0.12174479166666667, 0.12369791666666667, 0.12434895833333333, 0.125, 0.12565104166666666]
alpha =  3.0 computed
[0.12044270833333333, 0.12174479166666667, 0.12369791666666667, 0.12434895833333333, 0.125, 0.12565104166666666, 0.12565104166666666]
alpha =  3.5 computed
[0.12044270833333333, 0.12174479166666667, 0.12369791666666667, 0.12434895833333333, 0.125, 0.12565104166666666, 0.12565104166666666, 0.126953125]
alpha =  4.0 computed
[0.12044270833333333, 0.12174479166666667, 0.12369791666666667, 0.12434895833333333, 0.125, 0.12565104

alpha =  14.0 computed
[0.12044270833333333, 0.12174479166666667, 0.12369791666666667, 0.12434895833333333, 0.125, 0.12565104166666666, 0.12565104166666666, 0.126953125, 0.12630208333333334, 0.12630208333333334, 0.126953125, 0.12565104166666666, 0.12565104166666666, 0.12565104166666666, 0.12565104166666666, 0.12565104166666666, 0.12565104166666666, 0.12565104166666666, 0.126953125, 0.126953125, 0.126953125, 0.12760416666666666, 0.12760416666666666, 0.12760416666666666, 0.12760416666666666, 0.12760416666666666, 0.12825520833333334, 0.12825520833333334, 0.12825520833333334]
alpha =  14.5 computed
[0.12044270833333333, 0.12174479166666667, 0.12369791666666667, 0.12434895833333333, 0.125, 0.12565104166666666, 0.12565104166666666, 0.126953125, 0.12630208333333334, 0.12630208333333334, 0.126953125, 0.12565104166666666, 0.12565104166666666, 0.12565104166666666, 0.12565104166666666, 0.12565104166666666, 0.12565104166666666, 0.12565104166666666, 0.126953125, 0.126953125, 0.126953125, 0.12760416

alpha =  20.0 computed
[0.12044270833333333, 0.12174479166666667, 0.12369791666666667, 0.12434895833333333, 0.125, 0.12565104166666666, 0.12565104166666666, 0.126953125, 0.12630208333333334, 0.12630208333333334, 0.126953125, 0.12565104166666666, 0.12565104166666666, 0.12565104166666666, 0.12565104166666666, 0.12565104166666666, 0.12565104166666666, 0.12565104166666666, 0.126953125, 0.126953125, 0.126953125, 0.12760416666666666, 0.12760416666666666, 0.12760416666666666, 0.12760416666666666, 0.12760416666666666, 0.12825520833333334, 0.12825520833333334, 0.12825520833333334, 0.12825520833333334, 0.12825520833333334, 0.12825520833333334, 0.12825520833333334, 0.12825520833333334, 0.12890625, 0.12955729166666666, 0.12955729166666666, 0.12955729166666666, 0.13020833333333334, 0.13216145833333334, 0.13216145833333334]
alpha =  20.5 computed
[0.12044270833333333, 0.12174479166666667, 0.12369791666666667, 0.12434895833333333, 0.125, 0.12565104166666666, 0.12565104166666666, 0.126953125, 0.126302

alpha =  25.0 computed
[0.12044270833333333, 0.12174479166666667, 0.12369791666666667, 0.12434895833333333, 0.125, 0.12565104166666666, 0.12565104166666666, 0.126953125, 0.12630208333333334, 0.12630208333333334, 0.126953125, 0.12565104166666666, 0.12565104166666666, 0.12565104166666666, 0.12565104166666666, 0.12565104166666666, 0.12565104166666666, 0.12565104166666666, 0.126953125, 0.126953125, 0.126953125, 0.12760416666666666, 0.12760416666666666, 0.12760416666666666, 0.12760416666666666, 0.12760416666666666, 0.12825520833333334, 0.12825520833333334, 0.12825520833333334, 0.12825520833333334, 0.12825520833333334, 0.12825520833333334, 0.12825520833333334, 0.12825520833333334, 0.12890625, 0.12955729166666666, 0.12955729166666666, 0.12955729166666666, 0.13020833333333334, 0.13216145833333334, 0.13216145833333334, 0.13216145833333334, 0.1328125, 0.13346354166666666, 0.13346354166666666, 0.13346354166666666, 0.1328125, 0.13346354166666666, 0.13346354166666666, 0.13346354166666666, 0.1334635

alpha =  29.0 computed
[0.12044270833333333, 0.12174479166666667, 0.12369791666666667, 0.12434895833333333, 0.125, 0.12565104166666666, 0.12565104166666666, 0.126953125, 0.12630208333333334, 0.12630208333333334, 0.126953125, 0.12565104166666666, 0.12565104166666666, 0.12565104166666666, 0.12565104166666666, 0.12565104166666666, 0.12565104166666666, 0.12565104166666666, 0.126953125, 0.126953125, 0.126953125, 0.12760416666666666, 0.12760416666666666, 0.12760416666666666, 0.12760416666666666, 0.12760416666666666, 0.12825520833333334, 0.12825520833333334, 0.12825520833333334, 0.12825520833333334, 0.12825520833333334, 0.12825520833333334, 0.12825520833333334, 0.12825520833333334, 0.12890625, 0.12955729166666666, 0.12955729166666666, 0.12955729166666666, 0.13020833333333334, 0.13216145833333334, 0.13216145833333334, 0.13216145833333334, 0.1328125, 0.13346354166666666, 0.13346354166666666, 0.13346354166666666, 0.1328125, 0.13346354166666666, 0.13346354166666666, 0.13346354166666666, 0.1334635

...

alpha =  95.0 computed
[0.12044270833333333, 0.12174479166666667, 0.12369791666666667, 0.12434895833333333, 0.125, 0.12565104166666666, 0.12565104166666666, 0.126953125, 0.12630208333333334, 0.12630208333333334, 0.126953125, 0.12565104166666666, 0.12565104166666666, 0.12565104166666666, 0.12565104166666666, 0.12565104166666666, 0.12565104166666666, 0.12565104166666666, 0.126953125, 0.126953125, 0.126953125, 0.12760416666666666, 0.12760416666666666, 0.12760416666666666, 0.12760416666666666, 0.12760416666666666, 0.12825520833333334, 0.12825520833333334, 0.12825520833333334, 0.12825520833333334, 0.12825520833333334, 0.12825520833333334, 0.12825520833333334, 0.12825520833333334, 0.12890625, 0.12955729166666666, 0.12955729166666666, 0.12955729166666666, 0.13020833333333334, 0.13216145833333334, 0.13216145833333334, 0.13216145833333334, 0.1328125, 0.13346354166666666, 0.13346354166666666, 0.13346354166666666, 0.1328125, 0.13346354166666666, 0.13346354166666666, 0.13346354166666666, 0.1334635

alpha =  96.5 computed
[0.12044270833333333, 0.12174479166666667, 0.12369791666666667, 0.12434895833333333, 0.125, 0.12565104166666666, 0.12565104166666666, 0.126953125, 0.12630208333333334, 0.12630208333333334, 0.126953125, 0.12565104166666666, 0.12565104166666666, 0.12565104166666666, 0.12565104166666666, 0.12565104166666666, 0.12565104166666666, 0.12565104166666666, 0.126953125, 0.126953125, 0.126953125, 0.12760416666666666, 0.12760416666666666, 0.12760416666666666, 0.12760416666666666, 0.12760416666666666, 0.12825520833333334, 0.12825520833333334, 0.12825520833333334, 0.12825520833333334, 0.12825520833333334, 0.12825520833333334, 0.12825520833333334, 0.12825520833333334, 0.12890625, 0.12955729166666666, 0.12955729166666666, 0.12955729166666666, 0.13020833333333334, 0.13216145833333334, 0.13216145833333334, 0.13216145833333334, 0.1328125, 0.13346354166666666, 0.13346354166666666, 0.13346354166666666, 0.1328125, 0.13346354166666666, 0.13346354166666666, 0.13346354166666666, 0.1334635

alpha =  98.0 computed
[0.12044270833333333, 0.12174479166666667, 0.12369791666666667, 0.12434895833333333, 0.125, 0.12565104166666666, 0.12565104166666666, 0.126953125, 0.12630208333333334, 0.12630208333333334, 0.126953125, 0.12565104166666666, 0.12565104166666666, 0.12565104166666666, 0.12565104166666666, 0.12565104166666666, 0.12565104166666666, 0.12565104166666666, 0.126953125, 0.126953125, 0.126953125, 0.12760416666666666, 0.12760416666666666, 0.12760416666666666, 0.12760416666666666, 0.12760416666666666, 0.12825520833333334, 0.12825520833333334, 0.12825520833333334, 0.12825520833333334, 0.12825520833333334, 0.12825520833333334, 0.12825520833333334, 0.12825520833333334, 0.12890625, 0.12955729166666666, 0.12955729166666666, 0.12955729166666666, 0.13020833333333334, 0.13216145833333334, 0.13216145833333334, 0.13216145833333334, 0.1328125, 0.13346354166666666, 0.13346354166666666, 0.13346354166666666, 0.1328125, 0.13346354166666666, 0.13346354166666666, 0.13346354166666666, 0.1334635

alpha =  99.5 computed
[0.12044270833333333, 0.12174479166666667, 0.12369791666666667, 0.12434895833333333, 0.125, 0.12565104166666666, 0.12565104166666666, 0.126953125, 0.12630208333333334, 0.12630208333333334, 0.126953125, 0.12565104166666666, 0.12565104166666666, 0.12565104166666666, 0.12565104166666666, 0.12565104166666666, 0.12565104166666666, 0.12565104166666666, 0.126953125, 0.126953125, 0.126953125, 0.12760416666666666, 0.12760416666666666, 0.12760416666666666, 0.12760416666666666, 0.12760416666666666, 0.12825520833333334, 0.12825520833333334, 0.12825520833333334, 0.12825520833333334, 0.12825520833333334, 0.12825520833333334, 0.12825520833333334, 0.12825520833333334, 0.12890625, 0.12955729166666666, 0.12955729166666666, 0.12955729166666666, 0.13020833333333334, 0.13216145833333334, 0.13216145833333334, 0.13216145833333334, 0.1328125, 0.13346354166666666, 0.13346354166666666, 0.13346354166666666, 0.1328125, 0.13346354166666666, 0.13346354166666666, 0.13346354166666666, 0.1334635

alpha =  0.0 computed
alpha =  0.5 computed
alpha =  1.0 computed
alpha =  1.5 computed
alpha =  2.0 computed
alpha =  2.5 computed
alpha =  3.0 computed
alpha =  3.5 computed
alpha =  4.0 computed
alpha =  4.5 computed
alpha =  5.0 computed
alpha =  5.5 computed
alpha =  6.0 computed
alpha =  6.5 computed
alpha =  7.0 computed
alpha =  7.5 computed
alpha =  8.0 computed
alpha =  8.5 computed
alpha =  9.0 computed
alpha =  9.5 computed
alpha =  10.0 computed
alpha =  10.5 computed
alpha =  11.0 computed
alpha =  11.5 computed
alpha =  12.0 computed
alpha =  12.5 computed
alpha =  13.0 computed
alpha =  13.5 computed
alpha =  14.0 computed
alpha =  14.5 computed
alpha =  15.0 computed
alpha =  15.5 computed
alpha =  16.0 computed
alpha =  16.5 computed
alpha =  17.0 computed
alpha =  17.5 computed
alpha =  18.0 computed
alpha =  18.5 computed
alpha =  19.0 computed
alpha =  19.5 computed
alpha =  20.0 computed
alpha =  20.5 computed
alpha =  21.0 computed
alpha =  21.5 computed
alpha = 

In [7]:
# Q2 Gaussian Naive Bayes

# compute mean and standard deviation for ML estimation
xtrain_log_mean_spam = np.mean(xtrain_log[ytrain == 1], axis = 0)
xtrain_log_std_spam = np.std(xtrain_log[ytrain == 1], axis = 0)
xtrain_log_mean_nonspam = np.mean(xtrain_log[ytrain == 0], axis = 0)
xtrain_log_std_nonspam = np.std(xtrain_log[ytrain == 0], axis = 0)

In [8]:
def classify_gaussian(email):
    feature_count = len(email)
    
    param_spam = 0
    param_non_spam = 0
    
    for i in np.arange(feature_count):
        param_spam += -np.log(xtrain_log_std_spam[i]) - ((email[i] - xtrain_log_mean_spam[i])/(xtrain_log_std_spam[i])) ** 2 / 2
        param_non_spam += -np.log(xtrain_log_std_nonspam[i]) - ((email[i] - xtrain_log_mean_nonspam[i])/(xtrain_log_std_nonspam[i])) ** 2 / 2
        
    prob_spam = MLE + param_spam
    prob_non_spam = MLE_neg + param_non_spam
    
    return (prob_spam > prob_non_spam).astype('uint8')

def error_rate_gaussian(data, result):
    error_count = 0
    
    for count, email in enumerate(data):
        if (classify_gaussian(email) != result[count]):
            error_count += 1
    return (error_count / len(data)) * 100

In [9]:
error_rate_gaussian_train = error_rate_gaussian(xtrain_log, ytrain)
error_rate_gaussian_test = error_rate_gaussian(xtest_log, ytest)
print (error_rate_gaussian_train)
print (error_rate_gaussian_test)

0.16867862969004893
0.16341145833333334


In [10]:
# Q3 Logistic Regression
lambdas = np.hstack((np.arange(1,10,1), np.arange(10,105,5)))

In [41]:
def sigmoid(x):
    return 1/(1+np.exp(-x))

def calc_parameters(x_data, y_data, omega, lam):
    feature_count = x_data.shape[1]
    x_data = np.insert(x_data, 0, 1, axis = 1)
    I = np.diag(np.insert(np.ones(feature_count), 0, 0))
    NLL = 0
    
    mus = sigmoid(np.dot(x_data, omega)) # an array of mu for each individual email in x_train
    s_param = []
    
    for y, mu in zip(y_data, mus):
        NLL -= y * np.log(mu) + (1 - y) * np.log(1 - mu)
        s_param.append(mu * (1 - mu))
    g = np.dot(x_data.T, (mus - y_data))
    S = np.diag(s_param)
    H = np.dot(x_data.T, np.dot(S, x_data))
    
    # regularization of parameters
#     NLL_reg = NLL + 0.5 * lam * (np.dot(omega.T, omega))
    NLL_reg = NLL + 0.5 * lam * np.inner(omega[1:], omega[1:])
    g_reg = g + np.insert(lam * omega[1:], 0, 0)
    H_reg = H + np.dot(lam, I)
    
    return NLL_reg, g_reg, H_reg

def calc_omega(x_data, y_data, lam):
    feature_count = x_data.shape[1]
    Init = np.zeros(feature_count + 1)
    omega = Init
    convergence = False
    iter_count = 0
    
    try:
        while convergence == False:
            NLL, g, H = calc_parameters(x_data, y_data, omega, lam)
            iter_count += 1
            omega_new = omega - np.dot(np.linalg.inv(H), g)
            error = np.inner(omega_new - omega, omega_new - omega)/np.inner(omega, omega)
            if error < 0.001:
                convergence = True
            else:
                omega = omega_new
        return omega_new
    except:
        print ("lambda value: ", lam, "error: ", error * 100, "omega: ", omega_new)
        

def calc_error_rate_logReg(x_data, y_data, x_train, y_train, lam):
    omega = calc_omega(x_train, y_train, lam)
    error_count = 0
    classified = sigmoid(omega[0] + np.dot(x_data, omega[1:])) > 0.5
    for idx, item in enumerate(classified):
        if (item != y_data[idx]):
            error_count += 1
    return (error_count / len(y_data)) * 100


In [43]:
logReg_train_error_rate = []
valid_lambda_train = []
invalid_lambda_train = []

for lam in lambdas:
    try:
        logReg_train_error_rate.append(calc_error_rate_logReg(xtrain_log, ytrain, xtrain_log, ytrain, lam))
        valid_lambda_train.append(lam)
    except:
        invalid_lambda_train.append(lam)
        pass

logReg_test_error_rate = []
valid_lambda_test = []
invalid_lambda_test = []

for lam in lambdas:
    try:
        logReg_test_error_rate.append(calc_error_rate_logReg(xtest_log, ytest, xtrain_log, ytrain, lam))
        valid_lambda_test.append(lam)
    except:
        invalid_lambda_test.append(lam)
        pass
print (logReg_train_error_rate)
print (valid_lambda_train)
print (invalid_lambda_train)

print (logReg_test_error_rate)
print (valid_lambda_test)
print (invalid_lambda_test)

D:\Anaconda\lib\site-packages\ipykernel_launcher.py:40: RuntimeWarning: divide by zero encountered in double_scalars


[0.053507340946166396, 0.053507340946166396, 0.053507340946166396, 0.053833605220228384, 0.05415986949429037, 0.05415986949429037, 0.05415986949429037, 0.053507340946166396, 0.053833605220228384, 0.053833605220228384, 0.054812398042414355, 0.05644371941272431, 0.055791190864600326, 0.05644371941272431, 0.05774877650897227, 0.05807504078303426, 0.058401305057096245, 0.05905383360522023, 0.05905383360522023, 0.05970636215334421, 0.05970636215334421, 0.06068515497553018, 0.06133768352365416, 0.06166394779771615, 0.06199021207177814, 0.0629690048939641, 0.06264274061990212, 0.06394779771615008]
[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 15, 20, 25, 30, 35, 40, 45, 50, 55, 60, 65, 70, 75, 80, 85, 90, 95, 100]
[]
[0.060546875, 0.060546875, 0.059244791666666664, 0.057942708333333336, 0.057291666666666664, 0.057942708333333336, 0.05859375, 0.057291666666666664, 0.05859375, 0.059895833333333336, 0.060546875, 0.0625, 0.061197916666666664, 0.0625, 0.06380208333333333, 0.06380208333333333, 0.0638020833333333

In [10]:
# Q4 K-Nearest Neighbors
Ks = np.hstack((np.arange(1,10,1), np.arange(10,105,5)))

In [11]:
def calc_Euclidean_distance (data_1, data_2):
#     Euc_dist = []
#     for index_1 in np.arange(len(data_1)):
#         euc_dist_temp = []
#         for index_2 in np.arange(len(data_2)):
#             euc_dist_temp.append(dist.euclidean(data_1[index_1], data_2[index_2]))
#         Euc_dist.append(euc_dist_temp)
#     return Euc_dist
    Euc_dist = np.array([[dist.euclidean(data_1[idx1], data_2[idx2]) for idx2 in np.arange(len(data_2))] for idx1 in np.arange(len(data_1))])
    return Euc_dist

def knn_error_rate(distance, y_data, y_train, K):
    error_count = 0
    for idx, distance_1D in enumerate(distance):
        dist_idx = np.argsort(distance_1D)[:K]
        spammed = (np.sum(y_train[dist_idx] == 1))
        non_spammed = (np.sum(y_train[dist_idx] == 0))
        if ((spammed > non_spammed) != y_data[idx]):
            error_count += 1
    return (error_count / len(y_data)) * 100
            

In [12]:
euc_dist_train = calc_Euclidean_distance(xtrain_log, xtrain_log)
euc_dist_test = calc_Euclidean_distance(xtest, xtrain)

knn_error_rate_test = []
knn_error_rate_train = []

for k in Ks:
    knn_error_rate_test.append(knn_error_rate(euc_dist_test, ytest, ytrain, k))
    knn_error_rate_train.append(knn_error_rate(euc_dist_train, ytrain, ytrain, k))
print (knn_error_rate_test)
print (knn_error_rate_train)

[8.463541666666668, 9.1796875, 7.942708333333333, 8.138020833333332, 8.333333333333332, 8.268229166666668, 8.203125, 8.3984375, 8.463541666666668, 8.854166666666668, 8.3984375, 9.440104166666668, 9.635416666666668, 10.7421875, 10.481770833333332, 11.067708333333332, 11.197916666666668, 11.1328125, 10.9375, 11.328125, 11.263020833333332, 11.263020833333332, 11.5234375, 11.71875, 11.848958333333332, 11.653645833333332, 11.653645833333332, 11.71875]
[0.03262642740619902, 3.066884176182708, 3.4257748776508974, 4.143556280587276, 4.241435562805873, 4.926590538336052, 4.698205546492659, 5.089722675367047, 4.632952691680261, 5.383360522022838, 6.068515497553018, 6.492659053833605, 6.753670473083197, 7.210440456769984, 7.308319738988581, 7.569331158238173, 7.765089722675366, 7.99347471451876, 8.05872756933116, 7.99347471451876, 8.287112561174553, 8.64600326264274, 8.67862969004894, 8.874388254486133, 8.841761827079935, 8.809135399673735, 8.776508972267537, 8.841761827079935]


In [12]:
print (calc_Euclidean_distance(xtrain_log, xtrain_log))

[[ 0.          7.5737939   6.81623814 ...  8.99716092  8.4855286
   7.7188345 ]
 [ 7.5737939   0.          7.57140277 ...  8.83353652  9.24550958
  10.23473476]
 [ 6.81623814  7.57140277  0.         ...  8.13892882  8.26819846
   6.81062845]
 ...
 [ 8.99716092  8.83353652  8.13892882 ...  0.          8.50056822
   9.27673152]
 [ 8.4855286   9.24550958  8.26819846 ...  8.50056822  0.
  10.21475828]
 [ 7.7188345  10.23473476  6.81062845 ...  9.27673152 10.21475828
   0.        ]]


array([0, 1, 2], dtype=int64)